# Libraries

In [50]:
import pickle as pkl
import pandas as pd

In [51]:
algo_name = "MF"
data_strategy = "fairbook"  # change depending on which dataset we want to examine

# Analysis

## LKPY

In [52]:
algo_versions = {"MF": [{"bias": True}, {"bias": False}]}

In [53]:
versions = algo_versions[algo_name]

In [54]:
file_location = "experimental_results/" + algo_name + "/"

In [55]:
results = []
# for data_strategy in data_strategies:
for args in versions:
    file = open(file_location + data_strategy + "_" + str(args) + ".pkl", "rb")
    result = pkl.load(file)
    results.append(result)

In [56]:
stringed_versions = [str(args) for args in versions]

In [57]:
# Initialize empty lists for the two halves
bias = []

# Split the strings and populate the lists
for s in stringed_versions:
    parts = s.split(": ")

    bias.append(parts[-1].split("}")[0])

In [58]:
library = ["Lenskit"]

In [59]:
index = pd.MultiIndex.from_product(
    [bias, library], names=["Bias", "Library"]
).drop_duplicates()

In [60]:
results_lkpy = pd.DataFrame(results, index=index)

## Cornac

In [61]:
algo_versions = {"CornacMF": [{"bias": True}, {"bias": False}]}
algo_name = "CornacMF"

In [62]:
versions = algo_versions[algo_name]

In [63]:
file_location = "experimental_results/" + algo_name + "/"

In [64]:
results = []
# for data_strategy in data_strategies:
for args in versions:
    file = open(file_location + data_strategy + "_" + str(args) + ".pkl", "rb")
    result = pkl.load(file)
    results.append(result)

In [65]:
stringed_versions = [str(args) for args in versions]

In [66]:
# Initialize empty lists for the two halves
bias = []

# Split the strings and populate the lists
for s in stringed_versions:
    parts = s.split(": ")

    bias.append(parts[-1].split("}")[0])

In [67]:
library = ["Cornac"]

In [68]:
index = pd.MultiIndex.from_product(
    [bias, library], names=["Bias", "Library"]
).drop_duplicates()

In [69]:
results_cornac = pd.DataFrame(results, index=index)

In [70]:
results_cornac

,,pop_corr,RMSE,NDCG,ARP,ave_PL,ACLT,AggDiv
Bias,Library,,,,,,,
True,Cornac,"(0.007876063179982292, 0.5123894845456104)",1.312559,0.009397,0.002267,-37.563355,2.512494,0.251698
False,Cornac,"(-0.01381451360101617, 0.250509873109233)",1.460234,0.000993,0.001456,-61.163727,6.490933,0.173385


# Merge

In [71]:
mf_metrics = (
    pd.concat([results_lkpy, results_cornac])
    .reset_index()
    .sort_values(["Library", "Bias"])
    .set_index(["Library", "Bias"])
)

In [72]:
metrics_order = ["pop_corr", "ARP", "ave_PL", "ACLT", "AggDiv", "RMSE", "NDCG"]
mf_metrics = mf_metrics[metrics_order]

In [73]:
mf_metrics = mf_metrics.rename(
    columns={"pop_corr": "PopCorr", "ave_PL": "PL", "ACLT": "APLT", "NDCG": "NDCG@10"}
)

In [74]:
print(mf_metrics.drop("APLT", axis=1).round(3).to_latex())

\begin{tabular}{lllrrrrr}
\toprule
        &      &                                        PopCorr &    ARP &      PL &  AggDiv &   RMSE &  NDCG@10 \\
Library & Bias &                                                &        &         &         &        &          \\
\midrule
Cornac & False &      (-0.01381451360101617, 0.250509873109233) &  0.001 & -61.164 &   0.173 &  1.460 &    0.001 \\
        & True &     (0.007876063179982292, 0.5123894845456104) &  0.002 & -37.563 &   0.252 &  1.313 &    0.009 \\
Lenskit & False &    (-0.015301384386933124, 0.2030869873493037) &  0.002 & -59.558 &   0.269 &  1.762 &    0.003 \\
        & True &  (0.10821060670720456, 1.7610865056903274e-19) &  0.005 &  46.628 &   0.248 &  1.560 &    0.004 \\
\bottomrule
\end{tabular}



In [76]:
import pickle

with open("metrics_combined/"+data_strategy+"_all_mf.pkl", "wb") as f:
    pickle.dump(mf_metrics.round(3), f)